Parte 1: Generación del conjunto de datos

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import os
import csv
import sklearn.metrics as metrics

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 56.6 MB/s 
     |████████████████████████████████| 67 kB 7.7 MB/s 
     |████████████████████████████████| 3.3 MB 52.1 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
data_raw = pd.read_excel('/content/lcp_single_train spanish CLEXIS2.xlsx')

In [ ]:
data_raw = data_raw[['sentences', 'token', 'complexity']]

In [ ]:
data_raw['sentences'] = data_raw['sentences'].astype(str)
data_raw['token'] = data_raw['token'].astype(str)

In [ ]:
data = pd.DataFrame(columns=['text', 'complexity'])
count = 0
for i in range(0, data_raw.shape[0]):
  data.loc[count] = [data_raw['sentences'][count].lower() + ' [SEP] ' + data_raw['token'][count].lower(), \
                    data_raw['complexity'][count]]
  count += 1

In [ ]:
data

,text,complexity
0,texto espiral principios básicos decidir qué p...,0.2
1,texto tenemos elementos de las comunicaciones ...,0.2
2,el valor por omisión es 1 que significa que or...,0.2
3,sql wildcards wildcards comodines son caracter...,0.2
4,nan [SEP] 0abcdh,0.2
...,...,...
18372,texto ahora la soberanía de los estados es un ...,0.2
18373,texto la soberanía de los estados denota derec...,0.2
18374,cuando haces una consulta de saldo el celular ...,0.2
18375,billones de transistores contados ósea los ele...,0.2


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

In [ ]:
# Preprocess, tokenize and vectorize texts and labels
def vectorize_text(s, max_length):

    # Unicode normalization
    #s = s.strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    # Remove undesired characters
    #s = re.sub(r'([.¿?()\'",;:$€])', r' \1 ', s)
    #s = re.sub(r"[^a-zA-Záéíóú.,!?;:()$€]+", r" ", s)

    # Text to tensor
    input_ids = tokenizer.encode(
      s,
      add_special_tokens=True,
      max_length=max_length,
      padding='longest', 
      truncation=True,
      return_tensors='np'
    )
    return input_ids[0]

In [ ]:
data['text_vec'] = data.apply(lambda r: vectorize_text(r['text'], 512), axis=1)

In [ ]:
data

,text,complexity,text_vec
0,texto espiral principios básicos decidir qué p...,0.2,"[4, 4194, 23292, 3551, 1568, 1574, 7112, 1041,..."
1,texto tenemos elementos de las comunicaciones ...,0.2,"[4, 4194, 1796, 4365, 1009, 1085, 5780, 1319, ..."
2,el valor por omisión es 1 que significa que or...,0.2,"[4, 1039, 3155, 1076, 19015, 6983, 1028, 1098,..."
3,sql wildcards wildcards comodines son caracter...,0.2,"[4, 1015, 30975, 30962, 1005, 11733, 1488, 590..."
4,nan [SEP] 0abcdh,0.2,"[4, 8973, 5, 1444, 1107, 26051, 30974, 5]"
...,...,...,...
18372,texto ahora la soberanía de los estados es un ...,0.2,"[4, 4194, 1388, 1032, 7219, 3274, 1009, 1067, ..."
18373,texto la soberanía de los estados denota derec...,0.2,"[4, 4194, 1032, 7219, 3274, 1009, 1067, 1659, ..."
18374,cuando haces una consulta de saldo el celular ...,0.2,"[4, 1351, 2919, 1091, 7251, 1009, 14477, 1039,..."
18375,billones de transistores contados ósea los ele...,0.2,"[4, 20338, 1009, 18517, 7729, 1084, 29936, 234..."


In [ ]:
# Separar los conjuntos de datos
data = data.sample(frac=1)
train_portion = 0.8
split_point = int(train_portion*len(data))
train_data, test_data =  data[:split_point], data[split_point:] 
print(len(train_data), 'train, ', len(test_data), 'test')

14701 train,  3676 test


In [ ]:
#Create dataset en pytorch
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        input_ids = torch.tensor(self.data.text_vec.iloc[index]).cpu()
        attention_mask = torch.ones([input_ids.size(0)]).cpu()
        targets = self.data.complexity.iloc[index]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': targets
         }
            
    def __len__(self):
        return self.len

In [ ]:
train_set, test_set = MyDataset(train_data), MyDataset(test_data)

In [ ]:
test_set.data

,text,complexity,text_vec
3350,texto ósea que este me va a contar me va a con...,0.2,"[4, 4194, 23436, 30956, 1041, 1277, 1094, 1453..."
5493,texto un grafo dirigido es aquel que tiene tod...,0.8,"[4, 4194, 1044, 27733, 30957, 7823, 1028, 2491..."
18341,texto en resumidas cuentas big data es un conj...,0.4,"[4, 4194, 1035, 30392, 30958, 5007, 9282, 1071..."
884,los apasionados tienen alta emocionalidad valo...,0.6,"[4, 1067, 27706, 30958, 2018, 3531, 11564, 113..."
11559,texto en la parte de administración tenemos 1 ...,0.2,"[4, 4194, 1035, 1032, 1514, 1009, 2651, 1105, ..."
...,...,...,...
3414,lo que me va a garantizar que el dato sea igua...,0.4,"[4, 1086, 1041, 1094, 1453, 1012, 4425, 1041, ..."
7635,pero cuando los trabajos a realizar van a nece...,0.2,"[4, 1195, 1351, 1067, 4517, 1012, 4267, 2403, ..."
4252,datos inconsistente o datos incorrectos y ento...,1.0,"[4, 2783, 10622, 6173, 1068, 2783, 19682, 3095..."
13145,texto por ello debe existir un sitio para alma...,0.6,"[4, 4194, 1076, 2601, 1899, 10597, 1044, 3277,..."


In [ ]:
train_set.data

,text,complexity,text_vec
5319,registros de memoria entonces unidad de contro...,0.4,"[4, 8224, 1009, 5589, 1611, 4205, 1009, 2512, ..."
8872,texto una vez que conocemos algo de lo que es ...,0.2,"[4, 4194, 1091, 1434, 1041, 9485, 1423, 1009, ..."
8801,texto y eso es algo irrenunciable por ti algo ...,1.0,"[4, 4194, 1040, 1250, 1028, 1423, 6727, 16424,..."
17847,texto por medio del van se puede saber si un p...,0.8,"[4, 4194, 1076, 2079, 1081, 2403, 1057, 1428, ..."
15620,texto dentro de la unidad de control está el r...,0.2,"[4, 4194, 2193, 1009, 1032, 4205, 1009, 2512, ..."
...,...,...,...
13493,texto ahora tenemos una organización política ...,0.2,"[4, 4194, 1388, 1796, 1091, 9829, 1105, 25008,..."
15565,texto un proceso ágil debe atacarse incrementa...,0.4,"[4, 4194, 1044, 2621, 26273, 1899, 7764, 1182,..."
7286,inversiones y finanzas [SEP] finanzas,0.6,"[4, 6228, 1040, 10484, 5, 10484, 5]"
5143,texto diseño de objetos así mismo existen algu...,0.2,"[4, 4194, 3253, 1465, 1009, 6997, 3840, 1665, ..."


In [ ]:
from transformers import BertForSequenceClassification, BertConfig
config = BertConfig.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
config.problem_type='regression'
config.num_labels=1
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', config=config)


Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

In [ ]:
from transformers import Trainer, TrainingArguments

def collate_batch(batch):
    """ Optimize memory by setting all vectors in batch to a length equal to max
        length found
    """
    
    def pad_sequence(in_tensor, max_size):
        """ Fill tensor with zeros up to max_size
        """
        out_tensor = np.zeros(max_size)
        out_tensor[:in_tensor.size(0)] = in_tensor.numpy()
        return out_tensor
    
    batch_inputs = []
    batch_attention_masks = []
    batch_targets = []

    max_size = max([ex['input_ids'].size(0) for ex in batch])
    for item in batch:
        batch_inputs.append(pad_sequence(item['input_ids'], max_size))
        batch_attention_masks.append(pad_sequence(item['attention_mask'], max_size))
        batch_targets.append(float(item['labels']))
    return {
        "input_ids": torch.tensor(batch_inputs, dtype=torch.long),
        "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
        "labels": torch.tensor(batch_targets, dtype=torch.float)
    }

class MyTrainer(Trainer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def get_train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        collate_fn=collate_batch
    )

  def get_eval_dataloader(self, eval_dataset):
    return DataLoader(
        self.eval_dataset,
        collate_fn=collate_batch
    )


In [ ]:
training_args = TrainingArguments(
    output_dir='output',
    evaluation_strategy='steps',
    eval_steps=100,
    num_train_epochs=2,
    remove_unused_columns=False,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 14701
  Num Epochs = 2
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 29402
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Step,Training Loss,Validation Loss
100,No log,0.052194
200,No log,0.088242
300,No log,0.209138
400,No log,0.055523
500,0.084200,0.053913
600,0.084200,0.078982
700,0.084200,0.050545
800,0.084200,0.075804
900,0.084200,0.056753
1000,0.069200,0.100066


***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
***** Running Evaluation *****
  Num examples = 3676
  Batch size = 1
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in 

RuntimeError: ignored